In [1]:
import cv2
import numpy as np

In [2]:
from matplotlib import pyplot as plt

In [3]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

## Testing basic H threshold

In [82]:
# img = cv2.imread("./image/img01.png")
cap = cv2.VideoCapture("./video/test.mp4")
ret,img = cap.read()

In [ ]:
show_images([("a",img)])

In [122]:
def segment(orig,params,al,bet,gm):
    h_mean,h_std,l_mean,l_std,s_mean,s_std = params
    hls_im = cv2.cvtColor(orig,cv2.COLOR_BGR2HLS)
    a = (hls_im[:,:,0]>(h_mean-2.5*h_std))
    b = (hls_im[:,:,0]<(h_mean+2.5*h_std))
#     h_mask = a*b
    h_mask = np.logical_not(a*b)
    a = (hls_im[:,:,1]>(l_mean-2.5*l_std))
    b = (hls_im[:,:,1]<(l_mean+2.5*l_std))
#     l_mask = a*b
    l_mask = np.logical_not(a*b)
    a = (hls_im[:,:,2]>(s_mean-2.5*s_std))
    b = (hls_im[:,:,2]<(s_mean+2.5*s_std))
#     s_mask = a*b
    s_mask = np.logical_not(a*b)
#     hls_mask = h_mask*l_mask*s_mask
    hls_mask = al*h_mask + bet*l_mask + gm*s_mask
#     him = np.copy(img_reg)
#     lim = np.copy(img_reg)
#     sim = np.copy(img_reg)
#     im = np.copy(img_reg)
#     him[h_mask==True]=255
#     lim[l_mask==True]=255
#     sim[s_mask==True]=255
#     im[hls_mask==True]=255
#     show_images([("orig",img),("im",im),("sim",sim),("lim",lim),("him",him)])
#     return np.logical_not(hls_mask)*1.0
    return hls_mask

In [83]:
hls_img = cv2.cvtColor(img,cv2.COLOR_BGR2HLS).astype(np.float32)
r = cv2.selectROI("Select window of background",img=img,fromCenter=False,showCrossair=False)
cv2.destroyAllWindows()
r = [int(x) for x in r]
region = hls_img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
img_region = img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
h_mean,l_mean,s_mean = np.mean(region,axis=(0,1))
h_std,l_std,s_std = np.std(region,axis=(0,1))
params = [h_mean,h_std,l_mean,l_std,s_mean,s_std]

In [123]:
WHITE = np.ones(img.shape,dtype=np.int8)*255

In [147]:
cap = cv2.VideoCapture("./video/test.mp4")
while(True):
    ret,img = cap.read()
    if not ret:
        break
    mask = np.expand_dims(segment(img,params,0.4,0.3,0.3),-1)
    mask[mask>0.15]=1.0
    mask[mask<0.15]=0.0
    new_img = (mask*img+(1-mask)*WHITE).astype(np.uint8)
#     show_images([("img",ans),("masked",new_img)])
    cv2.imshow("img",img)
    cv2.imshow("masked",new_img)
    ret = cv2.waitKey(1)
    if ret==27:
        break
cv2.destroyAllWindows()

In [143]:
mask = np.expand_dims(segment(img,params,0.4,0.5,0.1),-1)
show_images([("a",mask)])

In [145]:
show_images([("a",mask)])

In [144]:
mask[mask>0.2]=1.0